In [25]:
import pandas as pd
import numpy as np

# Pre-processing.
from sklearn.preprocessing import StandardScaler

# Dummy.
from sklearn.dummy import DummyRegressor

# Models.
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Metrics.
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Evaluating.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV


# Step 1: Load data


In [26]:
training_data = pd.read_parquet('./assets/training_set_v2.parquet')
test_data = pd.read_parquet('./assets/test_set_v2.parquet')
validation_data = pd.read_parquet('./assets/validation_set_v2.parquet')


# Step 2: Standardize data


In [27]:
# Make sure y isn't in X.
columns_to_drop = ['events', 'ItemKey', 'RWB_EFFECTIVE_DATE']
X_train = training_data.drop(columns=columns_to_drop, axis=1)
X_test = test_data.drop(columns=columns_to_drop, axis=1)
X_val = validation_data.drop(columns=columns_to_drop, axis=1)

In [28]:
# Standardize values within each column to have a mean=0 and std=1.
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_val_std = scaler.transform(X_val)

In [29]:
# Do not standardize y.
y_train = training_data['events']
y_test = test_data['events']
y_val = validation_data['events']


# Step 3: Instantiate dummy regressors


In [30]:
dummy_regressor_mean = DummyRegressor(strategy='mean')
dummy_regressor_median = DummyRegressor(strategy='median')
dummy_regressor_quantile = DummyRegressor(strategy='quantile', quantile=0.25)


# Step 4: Define models


In [31]:
models = {
    'Dummy Mean': dummy_regressor_mean,
    'Dummy Median': dummy_regressor_median,
    'Dummy Quantile': dummy_regressor_quantile,
    'Linear Regression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Ridge Regression': Ridge(),
    'Elastic Net Regression': ElasticNet(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor()
}


# Step 5: Evaluate each model


In [32]:
val_results = []
for model_name, model in models.items():
    model.fit(X_train_std, y_train)
    predictions_val = model.predict(X_val_std)

    mse_val = mean_squared_error(y_val, predictions_val)
    rmse_test = np.sqrt(mse_val)
    mae_val = mean_absolute_error(y_val, predictions_val)

    val_results.append([model_name, mse_val, rmse_test, mae_val])


# Step 6: Load results into a DataFrame


In [33]:
def bold_below_threshold(val):
    if val <= 1.41:
        return 'font-weight: bold'
    else:
        return ''

val_metrics_df = pd.DataFrame(
    val_results,
    columns=['Model', 'Validation Set MSE', 'Validation Set RMSE', 'Validation Set MAE']
)
val_metrics_df.set_index('Model').style.format(precision=2).applymap(bold_below_threshold)
# Not as good as Dummy Quantile, but superior to Dummy Median.
# Almost in between the two.
# MAEs much closer to Quantile vs. MSEs.

,Validation Set MSE,Validation Set RMSE,Validation Set MAE
Model,,,
Dummy Mean,3.78,1.94,1.80
Dummy Median,2.50,1.58,1.40
Dummy Quantile,1.24,1.11,0.78
Linear Regression,7261011912780375.00,85211571.47,563569.30
Lasso Regression,3.78,1.94,1.80
Ridge Regression,2.93,1.71,1.44
Elastic Net Regression,3.76,1.94,1.80
Decision Tree Regression,6.68,2.58,1.14
Random Forest Regression,2.46,1.57,1.12



___
# Hyperparameter tuning


In [34]:
# List of top 18 features by importance:
# selected_features = [
#     'Days Since Creation',
#     'Days Since Last Logon',
#     'BIOSReleaseAge',
#     'LastBootAge',
#     'avg_software_age',
#     'FreeSpace_GB',
#     'num_installed_programs',
#     'Outlookx86_addin_filesize',
#     'Outlookx64_addin_filesize',
#     'Excelx86_addin_filesize',
#     'PowerPointx86_addin_filesize',
#     'Wordx64_addin_filesize',
#     'has_cap_iq_add',
#     'has_factset_add',
#     'InstallAge',
#     'num_users',
#     'num_updates',
#     'Total RAM'
# ]

# List of top 12 features by importance:
selected_features = [
    'Days Since Creation',
    'avg_software_age',
    'FreeSpace_GB',
    'Outlookx86_addin_filesize',
    'Wordx64_addin_filesize',
    'Days Since Last Logon',
    'num_installed_programs',
    'Outlookx64_addin_filesize',
    'InstallAge',
    'LastBootAge',
    'Excelx86_addin_filesize',
    'has_cap_iq_add'
]

In [35]:
# Create trimmed datasets
X_train_trimmed = X_train[selected_features]
X_test_trimmed = X_test[selected_features]
X_val_trimmed = X_val[selected_features]

In [36]:
print(X_train_trimmed.shape)
print(X_test_trimmed.shape)
print(X_val_trimmed.shape)

(78592, 12)
(45722, 12)
(45723, 12)


In [37]:
# Define the custom scoring function
def weighted_mae_fun(y_true, y_pred):
    """Scores by WMAE for cross_val_score()

    # Errors for 0 num events are 0.5 times as important.
    # Errors for 1 num events are 1 times as important.
    # Errors for 2 or more num events are 3 times as important.
    """
    errors = np.abs(y_true - y_pred)
    sample_weights = np.where(y_true == 0, 0.5, np.where(y_true == 1, 1, 3))
    weighted_errors = sample_weights * errors
    weighted_mae_score = np.sum(weighted_errors) / np.sum(sample_weights)

    return np.mean(weighted_mae_score)


#### RF: RandomizedSearchCV


In [38]:
# Use the RandomizedSearchCV method on a RandomForestRegressor model to identify a subset of parameters which could be optimal.
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=600, num=5)]
max_features = ['sqrt', 'log2']  # Number of features to consider at every split

max_depth = [int(x) for x in np.linspace(50, 300, num=6)]  # Maximum number of levels in tree
max_depth.append(None) # Add None.

min_samples_split = [2, 3, 5, 10, 20, 40]  # Minimum number of samples required to split a node
min_samples_leaf = [1, 3, 5, 10, 20, 40, 60, 80]  # Minimum number of samples required at each leaf node
bootstrap = [True, False]  # Method of selecting samples for training each tree

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
for k, v in random_grid.items():
    print(k, v)

n_estimators [200, 300, 400, 500, 600]
max_features ['sqrt', 'log2']
max_depth [50, 100, 150, 200, 250, 300, None]
min_samples_split [2, 3, 5, 10, 20, 40]
min_samples_leaf [1, 3, 5, 10, 20, 40, 60, 80]
bootstrap [True, False]


In [39]:
def evaluate_wmae(model, x, y):
    """
    Extends weighted_mae_fun out one level by asking the model to create
    predictions off x vs. handing them directly to weighted_mae_fun as
    y_pred.
    """
    preds = model.predict(x)
    weighted_errors = weighted_mae_fun(y, preds)
    print('Weighted Mean Absolute Error: {:0.2f}.'.format(weighted_errors))
    return weighted_errors

In [40]:
base_rf = RandomForestRegressor(
    n_estimators=10,
    # random_state=42
)
base_rf.fit(X_train_trimmed, y_train)
base_accuracy = evaluate_wmae(base_rf, X_val_trimmed, y_val)

Weighted Mean Absolute Error: 1.21.


In [41]:
rf = RandomForestRegressor() # First create the base model to tune.
# Random search of parameters, using 3-fold cross validation.
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100, # Search across n_iter different random combinations.
    cv=3,
    scoring=make_scorer(weighted_mae_fun, greater_is_better=False),
    random_state=42,
    n_jobs=-1, # Use all available cores.
    verbose=2,
)
rf_random.fit(X_train_trimmed, y_train) # Fit the random search model
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate_wmae(best_random, X_val_trimmed, y_val)

In [ ]:
# Best RandomizedSearchCV() params for WMAE: 1.12, n_iter=100:
# {'n_estimators': 400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 5,
#  'max_features': 'log2',
#  'max_depth': 250,
#  'bootstrap': False}


#### RF: GridSearchCV


In [ ]:
param_grid = rf_random.best_params_.copy()
param_grid

In [42]:
param_grid = {'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 5,
 'max_features': 'log2',
 'max_depth': 250,
 'bootstrap': False
 }

In [43]:
def build_params(begin, end, amount, var):
    """
    'min_samples_split' must be greater than 1.
    'min_samples_leaf' must be greater than 0.

    :param begin: Place at which to begin.
    :param end: Place at which to stop.
    :param amount: By how much to shift for each iteration.
    :param var: Which variable within the param_grid to build values for.
    :return: A list of new values to enter into a dictionary that is passed to grid search.
    """
    if param_grid[var] == 1:
        return [1, 2, 3]
    elif var == 'min_samples_split' and param_grid[var] < 2:
        return [2, 3, 4]
    elif var == 'max_depth' and param_grid[var] is None:
        return [None]
    elif var == 'max_depth' and param_grid[var] < 30:
        return [10, 20, 30, 40, 50]
    else:
        new_list = [param_grid[var] - (amount * i) for i in range(begin, 0, -1)]
        new_list.append(param_grid[var])
        new_list.extend(param_grid[var] + (amount * i) for i in range(1, end))
        return new_list

# Take what RandomizedSearchCV found and expand the space around those variables to pass to GridSearchCV.
param_grid.update({
    'n_estimators': build_params(1, 2, 50, var='n_estimators'),
    'min_samples_split': build_params(0, 3, 3, var='min_samples_split'),
    'min_samples_leaf': build_params(0, 3, 3, var='min_samples_leaf'),
    'max_features': [param_grid['max_features']],
    'max_depth': build_params(1, 2, 10, var='max_depth'),
    'bootstrap': [param_grid['bootstrap']]
})

# param_grid['n_estimators'].extend([int(param_grid['n_estimators'][-1] * 1.5)]) # Add a relatively large value.
# param_grid['max_depth'].append(None)
param_grid

{'n_estimators': [350, 400, 450],
 'min_samples_split': [2, 5, 8],
 'min_samples_leaf': [5, 8, 11],
 'max_features': ['log2'],
 'max_depth': [240, 250, 260],
 'bootstrap': [False]}

In [ ]:
# 81 fits @ n_estimators=400 = 8m.
base_rf_regressor = RandomForestRegressor()
grid_search_rf = GridSearchCV(
    base_rf_regressor,
    param_grid,
    scoring=make_scorer(weighted_mae_fun, greater_is_better=False),
    cv=3,
    n_jobs=-1,
    verbose=2
)
grid_search_rf.fit(X_train_trimmed, y_train)
best_rf_model_tuned = grid_search_rf.best_estimator_
best_rf_model_tuned

In [ ]:
grid_search_rf.best_params_

In [44]:
# Final model:
final_model = {'bootstrap': False,
 'max_depth': 250,
 'max_features': 'log2',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 450}

In [ ]:
gridsearch_accuracy = evaluate_wmae(best_rf_model_tuned, X_val_trimmed, y_val)


#### Evaluate on held-out test set


In [45]:
rf_test = RandomForestRegressor(**final_model)
rf_test.fit(X_train_trimmed, y_train)
evaluate_wmae(rf_test, X_test_trimmed, y_test)

Weighted Mean Absolute Error: 1.11.


1.114725202563421

In [ ]:
# RJ 08/13: One-time run: Observe how well the model performs on the test set.
evaluate_wmae(best_rf_model_tuned, X_test_trimmed, y_test)


# Re-define models for WMAE test results


In [46]:
models = {
    'Dummy Mean': dummy_regressor_mean,
    'Dummy Median': dummy_regressor_median,
    'Dummy Quantile': dummy_regressor_quantile,
    'Linear Regression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Ridge Regression': Ridge(),
    'Elastic Net Regression': ElasticNet(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': rf_test,
    'Gradient Boosting Regression': GradientBoostingRegressor()
}


# Evaluate each model on the test set


In [49]:
test_results = []
for model_name, model in models.items():
    model.fit(X_train_trimmed, y_train)
    predictions_test = model.predict(X_test_trimmed)

    mse_test = mean_squared_error(y_test, predictions_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_test, predictions_test)
    wmae_test = evaluate_wmae(model, X_test_trimmed, y_test)

    test_results.append([model_name, mse_test, rmse_test, mae_test, wmae_test])

Weighted Mean Absolute Error: 1.66.
Weighted Mean Absolute Error: 1.40.
Weighted Mean Absolute Error: 1.05.
Weighted Mean Absolute Error: 1.55.
Weighted Mean Absolute Error: 1.60.
Weighted Mean Absolute Error: 1.55.
Weighted Mean Absolute Error: 1.60.
Weighted Mean Absolute Error: 1.27.
Weighted Mean Absolute Error: 1.11.
Weighted Mean Absolute Error: 1.37.



# Load results into a DataFrame


In [51]:
def bold_below_threshold(val):
    if val <= 1.3:
        return 'font-weight: bold'
    else:
        return ''

test_metrics_df = pd.DataFrame(
    test_results,
    columns=['Model', 'Test Set MSE', 'Test Set RMSE', 'Test Set MAE', 'Test Set WMAE']
)
test_metrics_df.set_index('Model').style.format(precision=2).applymap(bold_below_threshold)

,Test Set MSE,Test Set RMSE,Test Set MAE,Test Set WMAE
Model,,,,
Dummy Mean,3.68,1.92,1.81,1.66
Dummy Median,2.38,1.54,1.40,1.40
Dummy Quantile,1.10,1.05,0.78,1.05
Linear Regression,3.12,1.77,1.62,1.55
Lasso Regression,3.34,1.83,1.70,1.60
Ridge Regression,3.12,1.77,1.62,1.55
Elastic Net Regression,3.32,1.82,1.69,1.60
Decision Tree Regression,5.35,2.31,1.16,1.27
Random Forest Regression,1.98,1.41,1.10,1.11
